In [20]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib as mpl
from collections import OrderedDict
import scipy.optimize 
import mpl_toolkits.axisartist as axisartist
import os
import math
import cmath
import scipy.optimize as opt
import random
import operator
import copy
import sys
import scipy.integrate as integrate
import scipy.sparse as sp
from scipy import stats
from numpy.linalg import inv
from scipy import sparse
from timeit import default_timer as timer
from scipy.integrate import dblquad
import numba
from numba import jit
import pylab as pl

## 1D Evolution of initial wave function in some static potential V(x):

We consider 1D time-dependent Schrödinger equation 

$i \hbar \, \partial_t \psi(x,t)=-\frac{1}{2m} \partial_x^2\psi(x,t)+V(x)\psi(x,t)$,

where $\psi$ is a normalized wave-function of the system and $V(x)$ denotes the static potential in 1D. For simplicity we consider units where $\hbar=m=1$. The system size is $L=1$ and we will assume both periodic (PBC) and open (OBC) boundary conditions.

To numerically evolve the system we employ the Crank–Nicolson finite difference method

$\partial_t u(x,t)=F(u,x,t,\partial_x u, \partial_x^2 u) \longrightarrow \frac{1}{\delta t}(u_i^{n+1}-u_i^n)=\frac{1}{2}\left[F_i^{n+1}+F_{i}^{n}\right]$,

where $(x,t)\rightarrow(x_0 + i \delta x, t_0 + n\delta t)$ with $i,n=0,1,2,\ldots$. Here we introduced the shorthand notation $u_i^n=u(x_0+i\delta x, t_0 + n\delta t)$ and similarly for $F$.  

In our problem we have second order derivative that can be discretized as follows ($x_i=x_0+i\delta x, \, t_n=t_0+n \delta t$)

$\partial_x^2 \psi(x_i,t_n) \longrightarrow \frac{1}{(\delta t)^2}(\psi_{i+1}^{n}-2\psi_i^n+ \psi_{i-1}^{n})$.

In consequence, by the Crank-Nicolson scheme we have

$\frac{i}{\delta t}(\psi_i^{n+1}-\psi_n^t) = \frac{1}{4(\delta t)^2} (\psi_{i+1}^{n}-2\psi_i^n+ \psi_{i-1}^{n})+ \frac{1}{4(\delta t)^2} (\psi_{i+1}^{n+1}-2\psi_i^{n+1}+ \psi_{i-1}^{n+1}) +\frac{1}{2}V_i(\psi_i^{n}+\psi_i^{n+1})$.

Let us introduce the matrix $\mathbb{D_2}$ correponding to the second derivative and vector od $\psi$'s at a given $t$

$\mathbb{D_2}=\left(
\begin{array}{ccccccccc}
-2 & 1 & 0 & 0 & \ldots & 0 & b\\
1 & -2 & 1 & 0&  0 & \ldots & 0 \\
0 & 1 & -2 & 1 & 0 &\ldots  &  0 \\
&&&\ldots &&&  \\
0 & \ldots  & 0 & 1 & -2 & 1 & 0\\
0 & \ldots & 0 & 0 & 1 & -2 & 1\\
b & 0 & \ldots & 0 & 0 & 1 & -2
\end{array}
\right)
\qquad \& \qquad
\Psi^n=\left(
\begin{array}{ccc}
\psi_0^n \\
\psi_1^n \\ 
\psi_2^n \\
\psi_3^n \\
.\\
.\\
.
\end{array}
\right),
\qquad \mathrm{where} \quad
b=\left\{\begin{array}{lll}
1 & \mathrm{for} & \mathrm{PBC} \\ 
0 & \mathrm{for} & \mathrm{OBC}
\end{array}\right.
.
$

By a simple reorganization of terms and introducing $\mathbb{O}_\pm=\frac{i}{\delta t}\mathbb{1}\pm\frac{1}{4 (\delta t)^2}\mathbb{D_2}\mp\frac{1}{2}\mathrm{diag}(V_0,V_1, V_2, \ldots)$ we immediately find

$\Psi^{n+1}=\mathbb{O}_+^{-1} \mathbb{O}_-\Psi^n$

In [21]:
# define matrices O_+^-1 and O_-:
def InvOplus(vecV, dx, dt, b): # b = 0 or 1 for OBC and PBC respectively
    N = len(vecV)
    A = 1./(4*dx*dx)
    diag = np.ones([N])*A
    mat = sp.spdiags([diag*b, diag,(1j/dt-2*diag-vecV/2.) ,diag, diag*b],[-N+1,-1,0,1,N-1],N,N)
    M0 = np.array(mat.toarray(),dtype = np.complex128)
    return inv(M0)

def Ominus(vecV, dx, dt, b): # b = 0 or 1 for OBC and PBC respectively
    N = len(vecV)
    A = 1./(4*dx*dx)
    diag = -np.ones([N])*A
    mat = sp.spdiags([diag*b, diag,(1j/dt-2*diag+vecV/2.) ,diag, diag*b],[-N+1,-1,0,1,N-1],N,N)
    return mat

# calculate energy and norm of the state to track stability of the evolution
def energy_norm(psi, vecV, dx):
    ddxpsi=[ 1./(dx**2)*(psi[j+1]+psi[j-1]-2*psi[j]) for j in range(1,len(psi)-1)]
    K = -0.5*np.conjugate(psi[1:M-1])*ddxpsi
    U = vecV[1:M-1]*np.absolute(psi[1:M-1])**2
    return [np.sum(K+U)*dx, np.sum(np.absolute(psi)**2)*dx]
    

def evolution(IOp, Om, Nsaves, Nsteps, psi, vecV, dx, dt):
      
    states = np.zeros((Nsaves+1,M),dtype=np.complex128)
    states[0] = psi
    Tab_e_n = energy_norm(psi, vecV, dx)

    for s in range(0,Nsaves):
        for j in range(0,Nsteps):
            psi = IOp.dot( Om.dot(psi) )
        
        states[s+1] = psi
        e_n = energy_norm(psi, vecV, dx)
        Tab_e_n.append(e_n)
        print(
              "time = "+str(np.round((s+1)*Nsteps*dt,5) ) 
                +" , energy = " +str(np.round(e_n[0],8) ) 
                + " , norm = " +str(np.round(e_n[1],8) )
             )
    return [states, Tab_e_n]

In [38]:
L = 20
M = 400
X = np.linspace(-L/2,L/2,M)
dx = X[1]-X[0]
dt = 0.0001
b=0

# define potential 
# let us try harmonic trap plus small obstacle in the center
a=0.00
h=0
width = 0.
vecV = a*X**4 + h*(np.tanh((X+width)/0.01)-np.tanh((X-width)/0.01) )

vecV = 10*(np.tanh((X-8)/0.001)+1)-10*(np.tanh((X+8)/0.001)-1)

#a=0.02
#b=2./5
#x0=np.sqrt(b)/np.sqrt(2.*a)
#vecV = a*X**4-b*X**2 - ( a*x0**4-b*x0**2 )

# set initial state with a gaussian-shape density
mu = 0
std = 1
snd = stats.norm(mu, std)

plt.plot(X,vecV);
plt.plot(X, snd.pdf(X))
plt.ylim((0,1));
plt.show()

In [23]:
# set the state into a motion with velocity v
v=5
psi =  np.sqrt(snd.pdf(X))*np.exp(1j*v*X)

# generate matrices
IOp = InvOplus(vecV, dx, dt, b)
Om  = Ominus(vecV, dx, dt, b)

Nsteps = 250
Nsaves= 600
W = evolution(IOp, Om, Nsaves, Nsteps, psi, vecV, dx, dt)

time = 0.025 , energy = (12.55578157-0j) , norm = 1.0
time = 0.05 , energy = (12.55578157+0j) , norm = 1.0
time = 0.075 , energy = (12.55578157+0j) , norm = 1.0
time = 0.1 , energy = (12.55578157+0j) , norm = 1.0
time = 0.125 , energy = (12.55578157-0j) , norm = 1.0
time = 0.15 , energy = (12.55578157-0j) , norm = 1.0
time = 0.175 , energy = (12.55578157+0j) , norm = 1.0
time = 0.2 , energy = (12.55578157+0j) , norm = 1.0
time = 0.225 , energy = (12.55578157+0j) , norm = 1.0
time = 0.25 , energy = (12.55578157+0j) , norm = 1.0
time = 0.275 , energy = (12.55578157+0j) , norm = 1.0
time = 0.3 , energy = (12.55578157-0j) , norm = 1.0
time = 0.325 , energy = (12.55578157-0j) , norm = 1.0
time = 0.35 , energy = (12.55578157-0j) , norm = 1.0
time = 0.375 , energy = (12.55578157+0j) , norm = 1.0
time = 0.4 , energy = (12.55578157-0j) , norm = 1.0
time = 0.425 , energy = (12.55578157-0j) , norm = 1.0
time = 0.45 , energy = (12.55578157+0j) , norm = 1.0
time = 0.475 , energy = (12.55578157+0j) 

time = 4.05 , energy = (12.55577234-1.2e-07j) , norm = 1.0
time = 4.075 , energy = (12.5557722-1.5e-07j) , norm = 1.0
time = 4.1 , energy = (12.55577203-1.7e-07j) , norm = 1.0
time = 4.125 , energy = (12.55577184-1.9e-07j) , norm = 1.0
time = 4.15 , energy = (12.55577163-2.2e-07j) , norm = 1.0
time = 4.175 , energy = (12.5557714-2.4e-07j) , norm = 1.0
time = 4.2 , energy = (12.55577114-2.6e-07j) , norm = 1.0
time = 4.225 , energy = (12.55577088-2.7e-07j) , norm = 1.0
time = 4.25 , energy = (12.5557706-2.8e-07j) , norm = 1.0
time = 4.275 , energy = (12.55577032-2.8e-07j) , norm = 1.0
time = 4.3 , energy = (12.55577005-2.8e-07j) , norm = 1.0
time = 4.325 , energy = (12.55576979-2.7e-07j) , norm = 1.0
time = 4.35 , energy = (12.55576956-2.5e-07j) , norm = 1.0
time = 4.375 , energy = (12.55576935-2.2e-07j) , norm = 1.0
time = 4.4 , energy = (12.55576917-1.9e-07j) , norm = 1.0
time = 4.425 , energy = (12.55576904-1.5e-07j) , norm = 1.0
time = 4.45 , energy = (12.55576894-1.1e-07j) , norm = 

time = 7.65 , energy = (12.55577597+4e-08j) , norm = 1.0
time = 7.675 , energy = (12.55577598+6e-08j) , norm = 1.0
time = 7.7 , energy = (12.555776+7e-08j) , norm = 1.0
time = 7.725 , energy = (12.55577604+7e-08j) , norm = 1.0
time = 7.75 , energy = (12.55577608+6e-08j) , norm = 1.0
time = 7.775 , energy = (12.55577612+5e-08j) , norm = 1.0
time = 7.8 , energy = (12.55577615+4e-08j) , norm = 1.0
time = 7.825 , energy = (12.55577617+2e-08j) , norm = 1.0
time = 7.85 , energy = (12.55577617-0j) , norm = 1.0
time = 7.875 , energy = (12.55577616-3e-08j) , norm = 1.0
time = 7.9 , energy = (12.55577613-5e-08j) , norm = 1.0
time = 7.925 , energy = (12.55577608-6e-08j) , norm = 1.0
time = 7.95 , energy = (12.55577602-8e-08j) , norm = 1.0
time = 7.975 , energy = (12.55577596-9e-08j) , norm = 1.0
time = 8.0 , energy = (12.55577589-9e-08j) , norm = 1.0
time = 8.025 , energy = (12.55577582-9e-08j) , norm = 1.0
time = 8.05 , energy = (12.55577576-8e-08j) , norm = 1.0
time = 8.075 , energy = (12.55577

time = 11.625 , energy = (12.55577402+8e-08j) , norm = 1.0
time = 11.65 , energy = (12.55577411+7e-08j) , norm = 1.0
time = 11.675 , energy = (12.55577418+6e-08j) , norm = 1.0
time = 11.7 , energy = (12.55577424+5e-08j) , norm = 1.0
time = 11.725 , energy = (12.55577428+4e-08j) , norm = 1.0
time = 11.75 , energy = (12.55577431+3e-08j) , norm = 1.0
time = 11.775 , energy = (12.55577432+2e-08j) , norm = 1.0
time = 11.8 , energy = (12.55577432+0j) , norm = 1.0
time = 11.825 , energy = (12.5557743-1e-08j) , norm = 1.0
time = 11.85 , energy = (12.55577426-3e-08j) , norm = 1.0
time = 11.875 , energy = (12.55577421-5e-08j) , norm = 1.0
time = 11.9 , energy = (12.55577413-6e-08j) , norm = 1.0
time = 11.925 , energy = (12.55577404-8e-08j) , norm = 1.0
time = 11.95 , energy = (12.55577394-1e-07j) , norm = 1.0
time = 11.975 , energy = (12.55577383-1.1e-07j) , norm = 1.0
time = 12.0 , energy = (12.5557737-1.2e-07j) , norm = 1.0
time = 12.025 , energy = (12.55577358-1.2e-07j) , norm = 1.0
time = 12

<IPython.core.display.Javascript object>


In [39]:
# Enable interactive plot
%matplotlib notebook

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

fig, ax = plt.subplots()
line, = ax.plot([])     # A tuple unpacking to unpack the only plot
ax.set_xlim(-L/2,L/2)
ax.set_ylim(0, 0.8)
plt.plot(X,vecV,color='black')

def animate(frame_num):
    Y = np.absolute( W[0][frame_num] )**2
    line.set_data((X, Y))
    return line

anim = FuncAnimation(fig, animate, frames=len(W[0]), interval=20)
plt.show()